In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import csv
import random
import numpy as np
import tensorflow as tf
from google.cloud import storage
from ml.models.three_d import c3d
from etl.lib import cloud_management
from keras.preprocessing.image import ImageDataGenerator

/home/harold_triedman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [16]:
BLACKLIST = []
        
# Delete all content in tmp/npy/
filelist = [f for f in os.listdir('tmp/npy')]
for f in filelist:
    os.remove(os.path.join('tmp/npy', f))

# Access Google Cloud Storage
gcs_client = storage.Client.from_service_account_json(
    '/home/harold_triedman/elvo-analysis/credentials/client_secret.json'
)
bucket = gcs_client.get_bucket('elvos')

# Get label data from Google Cloud Storage
blob = storage.Blob('augmented_annotated_labels.csv', bucket)
blob.download_to_filename('tmp/augmented_annotated_labels.csv')
prelim_label_data = {}
with open('tmp/augmented_annotated_labels.csv', 'r') as pos_file:
    reader = csv.reader(pos_file, delimiter=',')
    for row in reader:
        if row[1] != 'Unnamed: 0':
            prelim_label_data[row[1]] = int(row[2])


In [17]:
# Get all of the positives from the label data
positive_label_data = {}
print('getting positive labels')
for id_, label in list(prelim_label_data.items()):
    if label == 1 and '_' in id_:
        positive_label_data[id_] = label

getting positive labels


In [18]:
# Get 12096 random negatives from the label data to feed into our generator
negative_counter = 0
negative_label_data = {}
print(f"getting {len(list(positive_label_data.keys()))} random negative labels")
while negative_counter < 6000:
    id_, label = random.choice(list(prelim_label_data.items()))
    if label == 0:
        negative_label_data[id_] = label
        del prelim_label_data[id_]
        negative_counter += 1

getting 12168 random negative labels


In [19]:
chunks = []
labels = []

In [20]:
i = 1
for id_, label in list(negative_label_data.items()):
    print(i, 'chunk_data/normal/negative/' + id_ + '.npy')
    blob = bucket.get_blob('chunk_data/normal/negative/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
        arr = np.expand_dims(arr, axis=-1)
        chunks.append(arr)
        labels.append(label)
        i += 1

1 chunk_data/normal/negative/8S0FHQGBA5LPLI4I417.npy
2 chunk_data/normal/negative/VKDRQPXRMRRBAHW3459.npy
3 chunk_data/normal/negative/IYDXJTFVWJEX36DO466.npy
4 chunk_data/normal/negative/HVPI7QQSXPDHSFME51.npy
5 chunk_data/normal/negative/ARZ7H7BSITHOSSMG62.npy
6 chunk_data/normal/negative/0KSBX96F8BU1FCFQ164.npy
7 chunk_data/normal/negative/H4C8M2RF0ASI8M5R468.npy
8 chunk_data/normal/negative/KEYZSDHDH2DINCKO206.npy
8 chunk_data/normal/negative/BCNW7CON2NV3XNKA170.npy
9 chunk_data/normal/negative/WPNYHBQRGJ4NHU3C39.npy
9 chunk_data/normal/negative/0DQO9A6UXUQHR8RA122.npy
9 chunk_data/normal/negative/IGIIMA3LABMNETMA56.npy
9 chunk_data/normal/negative/2ZF1T5U8LU7SMI4Y12.npy
10 chunk_data/normal/negative/GQ4W68F9T36QYU6B285.npy
11 chunk_data/normal/negative/5ONG9EN0RF9CYGFY140.npy
12 chunk_data/normal/negative/CPPRXSBJ6WA6OQ31166.npy
13 chunk_data/normal/negative/XJJ3IXEOPNFXBGP7731.npy
13 chunk_data/normal/negative/CCVIH3DIM3GF6SLG1460.npy
14 chunk_data/normal/negative/THOT71FJ6S8OS79

123 chunk_data/normal/negative/TT75ZBIGTHZ8MTD9145.npy
124 chunk_data/normal/negative/NTFYWO8XKLEW332D323.npy
125 chunk_data/normal/negative/FQJ4RD45AQ53MYRC626.npy
126 chunk_data/normal/negative/POSX6PS6LR34HENB301.npy
127 chunk_data/normal/negative/TBNR4PRU7XIL27G6373.npy
128 chunk_data/normal/negative/BHCMG7A3E5Y13T0J273.npy
129 chunk_data/normal/negative/Y4GJVIUVR4NAMLXW177.npy
130 chunk_data/normal/negative/I5LGG0BM4IP16DPT245.npy
131 chunk_data/normal/negative/YEP4A7J1PJN5BMDO170.npy
132 chunk_data/normal/negative/HXKDQ5I26Z2CRUQB532.npy
133 chunk_data/normal/negative/PCNMFAZL5VWWK7RP473.npy
134 chunk_data/normal/negative/PVMXD3RZSHEQTYW3707.npy
135 chunk_data/normal/negative/YDJX25T27CYATQ2539.npy
136 chunk_data/normal/negative/BSBXPIGXEUV3WWOZ170.npy
137 chunk_data/normal/negative/Z9F4JC7DA3ZIW46J182.npy
138 chunk_data/normal/negative/OTIP25YORHBKJPZM519.npy
139 chunk_data/normal/negative/MHL54TPCTOQJ2I4K663.npy
140 chunk_data/normal/negative/X63IP208CJ8ST0SX281.npy
141 chunk_d

258 chunk_data/normal/negative/EWYGH3OSPLJI27UF36.npy
259 chunk_data/normal/negative/6SFFLWAD7TCCTTHC53.npy
259 chunk_data/normal/negative/KZMPFXLAUCRM314K89.npy
260 chunk_data/normal/negative/XYAFZGBXWKQJN365346.npy
261 chunk_data/normal/negative/WBZY9S3WDDG0YL9Y934.npy
261 chunk_data/normal/negative/0P43IADQS0BGHJLC8.npy
262 chunk_data/normal/negative/Q9K5IEJR20DGZMSZ550.npy
263 chunk_data/normal/negative/Z9SQUSY4XPPSP4ND570.npy
263 chunk_data/normal/negative/11R9AD0U2YKRA2CA723.npy
263 chunk_data/normal/negative/UZFAM45KRI3C7MGB267.npy
264 chunk_data/normal/negative/GCBJDIT55TRGHZIR478.npy
265 chunk_data/normal/negative/SZTHO00BX3ZAE4Y5330.npy
266 chunk_data/normal/negative/51V4PDJDQYRK6LD044.npy
267 chunk_data/normal/negative/X18DVJEHEBZK43JN732.npy
268 chunk_data/normal/negative/LO0IGD0MRX9BKV3896.npy
268 chunk_data/normal/negative/UYA5YJUAVC3SE6HV374.npy
269 chunk_data/normal/negative/TVCP433TKOVBCQTW76.npy
269 chunk_data/normal/negative/IBRRD5O4NGVPABDT653.npy
270 chunk_data/nor

382 chunk_data/normal/negative/ML05G1O6XMYNN1JE30.npy
383 chunk_data/normal/negative/6SFFLWAD7TCCTTHC927.npy
383 chunk_data/normal/negative/PSGF227LMZ56WM20435.npy
384 chunk_data/normal/negative/DDIKD2BEX8VMNNIQ707.npy
385 chunk_data/normal/negative/YNLMIJ5S24AYYZXN207.npy
386 chunk_data/normal/negative/PBR2STY62FZ3MQHZ225.npy
387 chunk_data/normal/negative/JOJU5LPDDBBP6LYG11.npy
388 chunk_data/normal/negative/TS4UPSJ92SZ1EITM114.npy
389 chunk_data/normal/negative/GFRJ7SPGIEJVXVRS2225.npy
390 chunk_data/normal/negative/GFFSQZ18XGOVEET0384.npy
391 chunk_data/normal/negative/56GX2GI8AGT0BIHN215.npy
391 chunk_data/normal/negative/ICWMS7OCXZLUTUFT45.npy
392 chunk_data/normal/negative/RATVCRHG4ZHLCOV3271.npy
393 chunk_data/normal/negative/H75X3XPB2CE9PXZL223.npy
393 chunk_data/normal/negative/WBU2NUH7X0X5J4RN264.npy
394 chunk_data/normal/negative/J3CQA68PDJ31TV8R490.npy
394 chunk_data/normal/negative/WKKADQ0Q6519IXWY95.npy
395 chunk_data/normal/negative/XHX3PCDOWEWDB34X240.npy
395 chunk_dat

503 chunk_data/normal/negative/2JMHB4LET9UMEJKR13.npy
504 chunk_data/normal/negative/V2TUYLWYQGFUJOXN290.npy
505 chunk_data/normal/negative/SMGWMDYTYR8ZB3F549.npy
506 chunk_data/normal/negative/5LGDWR05UU6NKX4Z61.npy
507 chunk_data/normal/negative/WI171X7KEOFWUIUO122.npy
508 chunk_data/normal/negative/HGJTE6ROVCUFUC4S202.npy
508 chunk_data/normal/negative/VVGO45TQNOASBLZM419.npy
509 chunk_data/normal/negative/EPSAH2BCBGV7WQHM77.npy
510 chunk_data/normal/negative/8TLM2DUBYEE2GDH0200.npy
511 chunk_data/normal/negative/27QV9Z81JZB09QPH273.npy
512 chunk_data/normal/negative/LT4EQ19PF206URYF604.npy
513 chunk_data/normal/negative/DDWAA4OECH5TXONN337.npy
514 chunk_data/normal/negative/ARZ7H7BSITHOSSMG314.npy
515 chunk_data/normal/negative/CSFRT7ES11I1Y3CF496.npy
516 chunk_data/normal/negative/UEIDAD48BPY82DY3434.npy
517 chunk_data/normal/negative/C5A4R35NQOHJOCDZ456.npy
518 chunk_data/normal/negative/S1MFJS5SSJ12LGEF93.npy
519 chunk_data/normal/negative/LZBLW6F81ERF8901532.npy
520 chunk_data/

...

4566 chunk_data/normal/negative/DSIFGAA3BMQKFE7A500.npy
4566 chunk_data/normal/negative/TT75ZBIGTHZ8MTD9609.npy
4567 chunk_data/normal/negative/WZABX1OKJSMRLG6E278.npy
4568 chunk_data/normal/negative/UAFTPLA47GA7WS2785.npy
4569 chunk_data/normal/negative/37ZPX63RET9P2QHM350.npy
4570 chunk_data/normal/negative/MYTHDP7XSELNJUBQ348.npy
4571 chunk_data/normal/negative/QJ9YG0LHX1SFOKLN668.npy
4572 chunk_data/normal/negative/HCA8NOIQCJL1EQLK380.npy
4573 chunk_data/normal/negative/XJLRFX244JGJZUUF463.npy
4574 chunk_data/normal/negative/3L337CU2FCNMNBXH96.npy
4575 chunk_data/normal/negative/DRT3F4ITWPCCBB6T418.npy
4576 chunk_data/normal/negative/IBRRD5O4NGVPABDT605.npy
4577 chunk_data/normal/negative/7W6NJWN9S0UF5A8N352.npy
4578 chunk_data/normal/negative/U82WU8SEKUCXCHZI63.npy
4578 chunk_data/normal/negative/NJPOTXDCGONCOU7C477.npy
4579 chunk_data/normal/negative/ZVRO2HKWM7JSS6IA322.npy
4580 chunk_data/normal/negative/2D2SR4DRRR4CCSQK463.npy
4581 chunk_data/normal/negative/SZKV26SJOHXJBPVX267

4698 chunk_data/normal/negative/BCYM2RGUQ2W0HKEH355.npy
4699 chunk_data/normal/negative/JNTL5ESEIA5FWMUQ114.npy
4700 chunk_data/normal/negative/TFLSOP3FLUBQDLL254.npy
4701 chunk_data/normal/negative/HIA2VPHI6ABMCQTV48.npy
4702 chunk_data/normal/negative/5FY8Y34MBL26F0EE102.npy
4703 chunk_data/normal/negative/VKDRQPXRMRRBAHW3440.npy
4703 chunk_data/normal/negative/CJDXGZHXAGH7QL3C25.npy
4704 chunk_data/normal/negative/DDPQNEBC34XCOF2U114.npy
4705 chunk_data/normal/negative/SLPTPLSMZYJEUEXK245.npy
4706 chunk_data/normal/negative/4TM6IQICWY502GS3809.npy
4707 chunk_data/normal/negative/BBKQNRFE6QHDDKTP38.npy
4707 chunk_data/normal/negative/3SFAQXXE8MAOTWYL715.npy
4708 chunk_data/normal/negative/3YQNRP7QKLAD6YN5482.npy
4709 chunk_data/normal/negative/IXMOCOA4Z88DGJ6E36.npy
4710 chunk_data/normal/negative/HB04152LQFDZPEBP552.npy
4711 chunk_data/normal/negative/AOY7EXEF3H62XKG4453.npy
4712 chunk_data/normal/negative/DZOMCPYXT1TY592W49.npy
4713 chunk_data/normal/negative/FALQ5UCC0FUIK32D421.np

4821 chunk_data/normal/negative/IAPTWF22G2YMU1P6308.npy
4822 chunk_data/normal/negative/SBLQEZAZEXCC76C6146.npy
4823 chunk_data/normal/negative/AAU7FDO320X6D4A2121.npy
4824 chunk_data/normal/negative/PKDK1R3JGEDSY9T2587.npy
4825 chunk_data/normal/negative/VOIUV38X58F3KY75136.npy
4826 chunk_data/normal/negative/NHXCOHZ4HH53NLQ6361.npy
4827 chunk_data/normal/negative/ASI5JI7DJLB3RKFS345.npy
4828 chunk_data/normal/negative/GYEUW4CVVSI3VBY923.npy
4829 chunk_data/normal/negative/KHXSOC18T31CFB18108.npy
4830 chunk_data/normal/negative/QTT2T0HPF52M8B7K451.npy
4831 chunk_data/normal/negative/BBKQNRFE6QHDDKTP312.npy
4832 chunk_data/normal/negative/OTIP25YORHBKJPZM143.npy
4832 chunk_data/normal/negative/JUM3FRXBYLTATYYH364.npy
4833 chunk_data/normal/negative/6QXS535HGCT3QU7Z864.npy
4834 chunk_data/normal/negative/EGIAP2GDM33PCW5A456.npy
4835 chunk_data/normal/negative/IBRRD5O4NGVPABDT290.npy
4836 chunk_data/normal/negative/XPBRBUB6YYHIWHVO209.npy
4837 chunk_data/normal/negative/WCSSH2NHDNNZIVML2

4956 chunk_data/normal/negative/VOPMJMGIDQQ75E6G319.npy
4956 chunk_data/normal/negative/EGIAP2GDM33PCW5A115.npy
4957 chunk_data/normal/negative/WMMP0B4WE4H12UL534.npy
4958 chunk_data/normal/negative/MEQKZCRF5TR6MHLY364.npy
4959 chunk_data/normal/negative/VCG2AJ44VIANYB101044.npy
4960 chunk_data/normal/negative/NXSJBZWIQX5RKNO3401.npy
4960 chunk_data/normal/negative/ECZZ9D6T0D9K48X6302.npy
4961 chunk_data/normal/negative/0OJARUOGIFH5TZKU538.npy
4962 chunk_data/normal/negative/QGH1NCRAPQ9QI8DG417.npy
4963 chunk_data/normal/negative/IKLKZYXDYTCCYUAZ198.npy
4964 chunk_data/normal/negative/YEXJFFF2ZK44FYCP303.npy
4965 chunk_data/normal/negative/RWKA32WBSVFB4MQF32.npy
4966 chunk_data/normal/negative/RIOHJ27BKA2I1K0T528.npy
4967 chunk_data/normal/negative/IAUKV5R644JZFD55365.npy
4968 chunk_data/normal/negative/ICWMS7OCXZLUTUFT695.npy
4969 chunk_data/normal/negative/D8C02RJJ2Z63XMFD494.npy
4970 chunk_data/normal/negative/MD1ZI5BSP1QU1CUR106.npy
4971 chunk_data/normal/negative/EYEDMZ3Z6OIKPERL3

5082 chunk_data/normal/negative/BLPHSKRODCJLHZZS401.npy
5083 chunk_data/normal/negative/DD4MRFEOI10TRMOF529.npy
5084 chunk_data/normal/negative/TLNUR0GUPFTO4Y73602.npy
5084 chunk_data/normal/negative/OGRRL3DQAJGEL0ZH130.npy
5085 chunk_data/normal/negative/TFLSOP3FLUBQDLL26.npy
5086 chunk_data/normal/negative/7CX50JMLP3MB8Y6X1937.npy
5087 chunk_data/normal/negative/TSX191EO453BJ6BQ23.npy
5087 chunk_data/normal/negative/ZXXMMCGK6ANRKLFD106.npy
5088 chunk_data/normal/negative/WZABX1OKJSMRLG6E34.npy
5089 chunk_data/normal/negative/S4TWFS0QGVDONGB240.npy
5090 chunk_data/normal/negative/TS4UPSJ92SZ1EITM312.npy
5091 chunk_data/normal/negative/MSS82V0V6T9P13CV84.npy
5092 chunk_data/normal/negative/SRKT7U91OM48G8XF668.npy
5093 chunk_data/normal/negative/TCD5M6UQD4FBL6NM498.npy
5094 chunk_data/normal/negative/OXRK4I6CTXCENAZV707.npy
5095 chunk_data/normal/negative/0QPSB9IO98216B98153.npy
5096 chunk_data/normal/negative/YCCJTT3KSZR8MYJK171.npy
5097 chunk_data/normal/negative/9UIAZ2U1711BN4IW178.n

In [23]:
i = 1
for id_, label in list(positive_label_data.items()):
    if i % 2 != 0:
        i += 1
        continue
    if i > 10500:
        break

    print(i, 'chunk_data/normal/positive/' + id_  + '.npy')
    blob = bucket.get_blob('chunk_data/normal/positive/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
        arr = np.expand_dims(arr, axis=-1)
        chunks.append(arr)
        labels.append(label)
        i += 1

2 chunk_data/normal/positive/04IOS24JP70LHBGB508_2.npy
4 chunk_data/normal/positive/04IOS24JP70LHBGB508_4.npy
6 chunk_data/normal/positive/04IOS24JP70LHBGB508_6.npy
8 chunk_data/normal/positive/04IOS24JP70LHBGB508_8.npy
10 chunk_data/normal/positive/04IOS24JP70LHBGB508_10.npy
12 chunk_data/normal/positive/04IOS24JP70LHBGB508_12.npy
14 chunk_data/normal/positive/04IOS24JP70LHBGB508_14.npy
16 chunk_data/normal/positive/04IOS24JP70LHBGB508_16.npy
18 chunk_data/normal/positive/04IOS24JP70LHBGB508_18.npy
20 chunk_data/normal/positive/04IOS24JP70LHBGB508_20.npy
22 chunk_data/normal/positive/04IOS24JP70LHBGB508_22.npy
24 chunk_data/normal/positive/04IOS24JP70LHBGB508_24.npy
26 chunk_data/normal/positive/0DQO9A6UXUQHR8RA340_2.npy
28 chunk_data/normal/positive/0DQO9A6UXUQHR8RA340_4.npy
30 chunk_data/normal/positive/0DQO9A6UXUQHR8RA340_6.npy
32 chunk_data/normal/positive/0DQO9A6UXUQHR8RA340_8.npy
34 chunk_data/normal/positive/0DQO9A6UXUQHR8RA340_10.npy
36 chunk_data/normal/positive/0DQO9A6UXUQHR

292 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_4.npy
294 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_6.npy
296 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_8.npy
298 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_10.npy
300 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_12.npy
302 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_14.npy
304 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_16.npy
306 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_18.npy
308 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_20.npy
310 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_22.npy
312 chunk_data/normal/positive/1H37S4G2MIXMQ3QS471_24.npy
314 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_2.npy
316 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_4.npy
318 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_6.npy
320 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_8.npy
322 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_10.npy
324 chunk_data/normal/positive/1IN3A0RW0UHOBHMA310_12.npy
326 chunk_data/normal

584 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_8.npy
586 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_10.npy
588 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_12.npy
590 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_14.npy
592 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_16.npy
594 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_18.npy
596 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_20.npy
598 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_22.npy
600 chunk_data/normal/positive/2WEAS2OYMHM0QP0M733_24.npy
602 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_2.npy
604 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_4.npy
606 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_6.npy
608 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_8.npy
610 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_10.npy
612 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_12.npy
614 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_14.npy
616 chunk_data/normal/positive/2WN4Y1MAS89PSM9P402_16.npy
618 chunk_data/norm

874 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_10.npy
876 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_12.npy
878 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_14.npy
880 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_16.npy
882 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_18.npy
884 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_20.npy
886 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_22.npy
888 chunk_data/normal/positive/3LR0UAB5II6XJ0W8468_24.npy
890 chunk_data/normal/positive/3LS918JYONAZ7EUN466_2.npy
892 chunk_data/normal/positive/3LS918JYONAZ7EUN466_4.npy
894 chunk_data/normal/positive/3LS918JYONAZ7EUN466_6.npy
896 chunk_data/normal/positive/3LS918JYONAZ7EUN466_8.npy
898 chunk_data/normal/positive/3LS918JYONAZ7EUN466_10.npy
900 chunk_data/normal/positive/3LS918JYONAZ7EUN466_12.npy
902 chunk_data/normal/positive/3LS918JYONAZ7EUN466_14.npy
904 chunk_data/normal/positive/3LS918JYONAZ7EUN466_16.npy
906 chunk_data/normal/positive/3LS918JYONAZ7EUN466_18.npy
908 chunk_data/nor

1156 chunk_data/normal/positive/5145LT5Z86A98W91311_4.npy
1158 chunk_data/normal/positive/5145LT5Z86A98W91311_6.npy
1160 chunk_data/normal/positive/5145LT5Z86A98W91311_8.npy
1162 chunk_data/normal/positive/5145LT5Z86A98W91311_10.npy
1164 chunk_data/normal/positive/5145LT5Z86A98W91311_12.npy
1166 chunk_data/normal/positive/5145LT5Z86A98W91311_14.npy
1168 chunk_data/normal/positive/5145LT5Z86A98W91311_16.npy
1170 chunk_data/normal/positive/5145LT5Z86A98W91311_18.npy
1172 chunk_data/normal/positive/5145LT5Z86A98W91311_20.npy
1174 chunk_data/normal/positive/5145LT5Z86A98W91311_22.npy
1176 chunk_data/normal/positive/5145LT5Z86A98W91311_24.npy
1178 chunk_data/normal/positive/53SO7XOB1AERHGDS524_2.npy
1180 chunk_data/normal/positive/53SO7XOB1AERHGDS524_4.npy
1182 chunk_data/normal/positive/53SO7XOB1AERHGDS524_6.npy
1184 chunk_data/normal/positive/53SO7XOB1AERHGDS524_8.npy
1186 chunk_data/normal/positive/53SO7XOB1AERHGDS524_10.npy
1188 chunk_data/normal/positive/53SO7XOB1AERHGDS524_12.npy
1190

...

9310 chunk_data/normal/positive/RRZGAIOEIS6WPXDT599_22.npy
9312 chunk_data/normal/positive/RRZGAIOEIS6WPXDT599_24.npy
9314 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_2.npy
9316 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_4.npy
9318 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_6.npy
9320 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_8.npy
9322 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_10.npy
9324 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_12.npy
9326 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_14.npy
9328 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_16.npy
9330 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_18.npy
9332 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_20.npy
9334 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_22.npy
9336 chunk_data/normal/positive/RSKIY1U4X5QAUAAK155_24.npy
9338 chunk_data/normal/positive/RTC5RXQHUXCSPNLN284_2.npy
9340 chunk_data/normal/positive/RTC5RXQHUXCSPNLN284_4.npy
9342 chunk_data/normal/positive/RTC5RXQHUXCSPNLN284_6.npy
9344

9590 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_14.npy
9592 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_16.npy
9594 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_18.npy
9596 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_20.npy
9598 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_22.npy
9600 chunk_data/normal/positive/SE3O4E4F4Z70C75B412_24.npy
9602 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_2.npy
9604 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_4.npy
9606 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_6.npy
9608 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_8.npy
9610 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_10.npy
9612 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_12.npy
9614 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_14.npy
9616 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_16.npy
9618 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_18.npy
9620 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_20.npy
9622 chunk_data/normal/positive/SMGWMDYTYR8ZB3F5218_22.npy
9

9872 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_8.npy
9874 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_10.npy
9876 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_12.npy
9878 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_14.npy
9880 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_16.npy
9882 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_18.npy
9884 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_20.npy
9886 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_22.npy
9888 chunk_data/normal/positive/TBNR4PRU7XIL27G6412_24.npy
9890 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_2.npy
9892 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_4.npy
9894 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_6.npy
9896 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_8.npy
9898 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_10.npy
9900 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_12.npy
9902 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_14.npy
9904 chunk_data/normal/positive/TCD5M6UQD4FBL6NM588_16.npy
99

10150 chunk_data/normal/positive/TS4UPSJ92SZ1EITM513_22.npy
10152 chunk_data/normal/positive/TS4UPSJ92SZ1EITM513_24.npy
10154 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_2.npy
10156 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_4.npy
10158 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_6.npy
10160 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_8.npy
10162 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_10.npy
10164 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_12.npy
10166 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_14.npy
10168 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_16.npy
10170 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_18.npy
10172 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_20.npy
10174 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_22.npy
10176 chunk_data/normal/positive/TSDXCC6X3M7PG91E532_24.npy
10178 chunk_data/normal/positive/TSZFE43KG3NQJR69507_2.npy
10180 chunk_data/normal/positive/TSZFE43KG3NQJR69507_4.npy
10182 chunk_data/normal/positive/TSZFE43KG3NQJ

10426 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_10.npy
10428 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_12.npy
10430 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_14.npy
10432 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_16.npy
10434 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_18.npy
10436 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_20.npy
10438 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_22.npy
10440 chunk_data/normal/positive/UGXVSPJLHJL6AHSW265_24.npy
10442 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_2.npy
10444 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_4.npy
10446 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_6.npy
10448 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_8.npy
10450 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_10.npy
10452 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_12.npy
10454 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_14.npy
10456 chunk_data/normal/positive/UI0XEE9RCAEFSYP2411_16.npy
10458 chunk_data/normal/positive/UI0XEE9RCAE

In [25]:
tmp = list(zip(chunks, labels))
random.shuffle(tmp)
chunks, labels = zip(*tmp)

# Split based on validation
print('splitting based on validation split')
X_train = np.asarray(chunks[int(len(chunks) * 0.1):])
y_train = np.asarray(labels[int(len(labels) * 0.1):])

X_val = np.asarray(chunks[:int(len(chunks) * 0.1)])
y_val = np.asarray(labels[:int(len(labels) * 0.1)])

splitting based on validation split


In [27]:
print(len(chunks), len(chunks[0]), len(chunks[0][0]), len(chunks[0][0][0]))

11233 32 32 32


In [28]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [29]:
from keras.optimizers import SGD
from keras.metrics import binary_accuracy, binary_crossentropy
from keras.callbacks import ModelCheckpoint

LEARN_RATE = 1e-5

In [30]:
model = c3d.C3DBuilder.build()
model.compile(optimizer=SGD(lr=LEARN_RATE, momentum=0.9, nesterov=True),
              loss={"out_class": "binary_crossentropy"},
#               loss='binary_crossentropy'
              metrics={"out_class": [binary_accuracy, binary_crossentropy]})
#              metrics=['accuracy'])

/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:24: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(pool_size=(2, 1, 1), strides=(2, 1, 1), padding="same")`
  strides=(2, 1, 1), border_mode="same")(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:27: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), activation="relu", name="conv1", strides=(1, 1, 1), padding="same")`
  subsample=(1, 1, 1))(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:30: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  name='pool1')(x)
/home/harold_triedman/elvo-analysis/ml/models/three_d/c3d.py:35: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), activation="relu", name="conv2", strides=(1, 1, 1), padding="same")`
  subsample=(1, 1, 1))(x)
/home/harold_triedman/elvo-

____________________________________________________________________________________________________________________________________________
Layer (type)                                                   Output Shape                                            Param #              
input_1 (InputLayer)                                           (None, 32, 32, 32, 1)                                   0                    
____________________________________________________________________________________________________________________________________________
average_pooling3d_2 (AveragePooling3D)                         (None, 16, 32, 32, 1)                                   0                    
____________________________________________________________________________________________________________________________________________
conv1 (Conv3D)                                                 (None, 16, 32, 32, 64)                                  1792                 
_____________

In [31]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(10110, 32, 32, 32, 1) (10110,)
(1123, 32, 32, 32, 1) (1123,)


In [32]:
mc_callback = ModelCheckpoint(filepath='tmp/c3d_weights.hdf5', verbose=1)
model.fit(
    x=X_train,
    y=y_train,
    epochs=200,
    batch_size=16,
    callbacks=[mc_callback],
    validation_data = (X_val, y_val),
    verbose=1)

Train on 10110 samples, validate on 1123 samples
Epoch 1/200
10110/10110 [==============================] - 42s 4ms/step - loss: 0.3847 - binary_accuracy: 0.8844 - binary_crossentropy: 0.3847 - val_loss: 0.1989 - val_binary_accuracy: 0.9341 - val_binary_crossentropy: 0.1989

Epoch 00001: saving model to tmp/c3d_weights.hdf5
Epoch 2/200
10110/10110 [==============================] - 42s 4ms/step - loss: 0.1311 - binary_accuracy: 0.9583 - binary_crossentropy: 0.1311 - val_loss: 0.1455 - val_binary_accuracy: 0.9564 - val_binary_crossentropy: 0.1455

Epoch 00002: saving model to tmp/c3d_weights.hdf5
Epoch 3/200
10110/10110 [==============================] - 42s 4ms/step - loss: 0.0858 - binary_accuracy: 0.9738 - binary_crossentropy: 0.0858 - val_loss: 0.1398 - val_binary_accuracy: 0.9537 - val_binary_crossentropy: 0.1398

Epoch 00003: saving model to tmp/c3d_weights.hdf5
Epoch 4/200
10110/10110 [==============================] - 42s 4ms/step - loss: 0.0640 - binary_accuracy: 0.9823 - binar

KeyboardInterrupt: 

In [54]:
import matplotlib.pyplot as plt

In [19]:
new_scans = []
new_labels = []

for i, (id_, label) in enumerate(list(positive_label_data.items())):
    if i >= 650:
        break
    if i % 25 != 0:
        continue
    blob = bucket.get_blob('chunk_data/normal/positive/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
#         arr[arr < 75] = 0
#         arr[arr > 400] = 0
#         axial = np.max(arr, axis=0)
#         coronal = np.max(arr, axis=1)
#         sag = np.max(arr,axis=2)
#         print(id_, label)
#         fig, ax = plt.subplots(1, 3, figsize=(6, 10))
#         ax[0].imshow(axial, interpolation='none')
#         ax[1].imshow(coronal, interpolation='none')
#         ax[2].imshow(sag, interpolation='none')
#         fig.tight_layout()
#         plt.show()
        arr = np.expand_dims(arr, axis=-1)
        new_scans.append(arr)
        new_labels.append(label)
        i += 1
        
i = 1
for id_, label in list(negative_label_data.items()):
    if i > 25:
        break
    blob = bucket.get_blob('chunk_data/normal/negative/' + id_ + '.npy')
    arr = cloud_management.download_array(blob)
    if arr.shape == (32, 32, 32):
        
#         print('hello')
#         axial = np.max(arr, axis=0)
#         coronal = np.max(arr, axis=1)
#         sag = np.max(arr,axis=2)
#         fig, ax = plt.subplots(1, 3, figsize=(6, 10))
#         ax[0].imshow(axial, interpolation='none')
#         ax[1].imshow(coronal, interpolation='none')
#         ax[2].imshow(sag, interpolation='none')
#         fig.tight_layout()
#         plt.show()
        arr = np.expand_dims(arr, axis=-1)
        new_scans.append(arr)
        new_labels.append(label)
        i += 1
    
tmp = list(zip(new_scans, new_labels))
random.shuffle(tmp)
new_scans, new_labels = zip(*tmp)

X_train = np.asarray(new_scans)
y_train = np.asarray(new_labels)

# Split based on validation
# print('splitting based on validation split')
# X_train = np.asarray(scans[int(len(scans) * 0.1):])
# y_train = np.asarray(labels[int(len(labels) * 0.1):])

# X_val = np.asarray(scans[:int(len(scans) * 0.1)])
# y_val = np.asarray(labels[:int(len(labels) * 0.1)])

In [20]:
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(51, 32, 32, 32, 1) (51,)
(2370, 32, 32, 32, 1) (2370,)


In [21]:
model.fit(
    x=X_train,
    y=y_train,
    epochs=200,
    batch_size=16,
#     callbacks=[mc_callback],
#     validation_data = (X_val, y_val),
    verbose=1)

Epoch 1/200
51/51 [==============================] - 0s 6ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 2/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 3/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 4/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 5/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 6/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 7/200
51/51 [==============================] - 0s 4ms/step - loss: 7.8149 - binary_accuracy: 0.5098 - binary_crossentropy: 7.8149
Epoch 8/200
51/51 [=============================

KeyboardInterrupt: 